In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import urllib.request
from scipy.stats import mode

In [ ]:
rules_file = open("dane/ClassificationRules.txt","r")
rules_data = rules_file.readlines()
rules = np.empty((266,9350))
for i in range(266):
     rules[i, :] = list(map(float, rules_data[i].split()))
        

expert_file = open("dane/ImageExpertReduced.txt","r")
expert = expert_file.readlines()
expert = np.array([list(map(float, expert[0].split()))])

print(expert.shape)
print(rules.shape)

In [ ]:
def random_individual(p,d):
    random_vec = np.random.rand(d)
    return (random_vec < p).astype(int)

def random_population(p,d,N):
    return np.array([random_individual(p,d) for i in range(N)])

In [ ]:
most_voted = lambda v: mode(v)[0]

def opt_f(x):
    x = x[np.newaxis,:]
    a = rules * x.T
    b = a[~(a==0).all(1)]  #remove 0 rows
    votes = np.array([most_voted(v) for v in b.T])
    return ((votes == expert).sum() / len(expert))

In [ ]:
def PBIL(d,N,t1,t2,t3,T):

    costs = np.zeros((3,T))
    bests = np.zeros((T,d))
    
    p = np.ones(d)
    p = p/2
    P = random_population(p,d,N)
    for t in range(T):
        val = np.array([opt_f(P[i]) for i in range(N)])
        x_best = P[val.argmax()]

        costs[0][t] = val.max()
        costs[1][t] = val.mean()
        costs[2][t] = val.min()

        bests[t] = x_best
        
        for i in range(N):
            p = p * (1 - t1) + x_best * t1

        for j in range(N):
            if np.random.rand() < t2:
                p = p * (1 - t3) + random_individual((np.ones(d)/2),d) * t3

        P = random_population(p,d,N)
        
        print(t)
    return (costs,bests)

In [ ]:
data = PBIL(266,50,0.045,0.015,0.02,900)